<span style='font-size:20.5px; letter_spacing: 1px'>The project is an AB-test. Project's goal: to determine any causal effect in users behaviour after being exposed to a <b>old_page</b> design or <b>new_page</b> design, which is captured by <b>converted</b> column</span>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

<h2>1. Let's take a look at the data. Figure out size of both control and treatment groups, possibly clean the data, convert columns into needed data types</h2>

In [2]:
#read data
df = pd.read_csv('ab_data.csv')

In [3]:
#overview
for column in df.columns:
    print(column)
    print(df[column].unique())

user_id
[851104 804228 661590 ... 734608 697314 715931]
timestamp
['2017-01-21 22:11:48.556739' '2017-01-12 08:01:45.159739'
 '2017-01-11 16:55:06.154213' ... '2017-01-22 11:45:03.439544'
 '2017-01-15 01:20:28.957438' '2017-01-16 12:40:24.467417']
group
['control' 'treatment']
landing_page
['old_page' 'new_page']
converted
[0 1]


<span><b>Ensure user ids do not dublicate</b><span>

In [4]:

counter = df['user_id'].value_counts()

#counter
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])
df =df.merge(valid_users, on='user_id')

In [5]:
counter

805339    2
754884    2
722274    2
783176    2
898232    2
         ..
642985    1
771499    1
923606    1
712675    1
715931    1
Name: user_id, Length: 290584, dtype: int64

In [6]:
#determine size of the groups
df['group'].value_counts()

treatment    143397
control      143293
Name: group, dtype: int64

In [7]:
#ration of the groups
len(df[df['group'] == 'treatment']) / len(df[df['group'] == 'control']) - 1

0.000725785628048925

In [8]:
#how long lasted the experiment 
print(df[df['group'] == 'treatment']['timestamp'].min())
print(df[df['group'] == 'treatment']['timestamp'].max())

2017-01-02 13:42:05.378582
2017-01-24 13:41:44.097174


In [9]:
#overview
df['timestamp'].describe()

count                         286690
unique                        286690
top       2017-01-21 22:11:48.556739
freq                               1
Name: timestamp, dtype: object

In [10]:
#convert timestamps into a proper datatype
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [11]:
#df['timestamp'].astype('str')

<h3>start of the test...
</h3>

In [12]:
#making a variable with controled and converted users in it
df_control_converted = df[(df['group'] == 'control') & (df['converted'] == 1)]

In [13]:
#making a variable with controled and non-converted users in it
df_control_unconverted = df[(df['group'] == 'control') & (df['converted'] == 0)]

In [14]:
#making variable with treated and converted users and treated + non-converted
df_treatment_converted = df[(df['group'] == 'treatment') & (df['converted'] == 1)]
df_treatment_unconverted = df[(df['group'] == 'treatment') & (df['converted'] == 0)]

In [15]:
#assesing the ratio of the control group (we ideally want groups to be symmetrical)
control_ratio = len(df_control_converted)/len(df_control_unconverted)

In [16]:
#assesing the ration of the treatment group
treatment_ratio = len(df_treatment_converted) / len(df_treatment_unconverted)

In [17]:
#treatment and control groups are almost symmetrical, which allows for correct analysis
treatment_ratio/control_ratio

0.9863367207661116

In [18]:
#creating a week column
import datetime
df['week'] = df['timestamp'].dt.isocalendar().week

In [19]:
#storing only control group
treatment_data = df.loc[df['group'] != 'control']

In [20]:
#checking the ratios
df.groupby('group')['converted'].value_counts()

group      converted
control    0            126073
           1             17220
treatment  0            126372
           1             17025
Name: converted, dtype: int64

<span style='font-size: 22px; color:purple;'>Transform the data into 2x2 matrix for chi test</span>

In [21]:
observed_values = pd.crosstab(df['group'], df['converted']).values
observed_values

array([[126073,  17220],
       [126372,  17025]], dtype=int64)

In [22]:
#ration of both groups
control_rate = 17723 / (17723 + 129479)
treatment_rate = 17512 / (129762 + 17514)

In [23]:
control_rate


0.12039917935897611

In [24]:
treatment_rate

0.11890599961976153

In [25]:
#importing the needed library for statistical analysis and tests
from scipy.stats import chi2_contingency, chi2
a = 0.05

In [26]:
#chi2 statistics and p value
chi2_statistics, p_value, dof, expected_values = chi2_contingency(observed_values, correction=False)

In [27]:
#display the statistics
print(chi2_statistics, p_value)

1.426794609399621 0.23228827305833816


In [28]:
#testing hypothesis
h_0 = 'no relationship'
h_1 = 'relationship exists'
if chi2_statistics > a:
    print(h_1, ':', 'chi2', chi2_statistics, 'p_value', p_value)
elif chi2_statistics < a:
    print(h_0, ':', 'chi2', chi2_statistics, 'p_value', p_value)

relationship exists : chi2 1.426794609399621 p_value 0.23228827305833816


In [29]:

critical_value = chi2.ppf(1 - a, dof)

critical_value
if critical_value > 3:
    print(f'critical value of {critical_value} is significant')

critical value of 3.841458820694124 is significant


<H1>2. Lets us look at the conversion rate broken down weekly</H1>

In [30]:
#for how long did the experiment last
start_time = pd.to_datetime(df['timestamp'].min())
end_time = pd.to_datetime(df['timestamp'].max())
data_duration_seconds = (end_time - start_time).total_seconds()
data_duration_days = data_duration_seconds / (24 * 3600)
print(f'data collected for {data_duration_days} days' )

data collected for 21.999873633414353 days


In [31]:
df

,user_id,timestamp,group,landing_page,converted,week
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,3
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,2
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,2
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,3
...,...,...,...,...,...,...
286685,751197,2017-01-03 22:28:38.630509,control,old_page,0,1
286686,945152,2017-01-12 00:51:57.078372,control,old_page,0,2
286687,734608,2017-01-22 11:45:03.439544,control,old_page,0,3
286688,697314,2017-01-15 01:20:28.957438,control,old_page,0,2


In [32]:
#let's break down into weeks
df['week'].value_counts()

2    91380
3    91056
1    83745
4    20509
Name: week, dtype: Int64

In [33]:
#calculating the ration between control and treatment groups week by week
n_f_w = 4
experiment_data = df[df['week'] <= n_f_w]

<span style= 'font-size: 23px; color: red;'>The corresponding values for chi statistics and p values by week can be calculating by changing the number in the <span style='color: black;'><b>n_f_w variable</b></span> from 1 to 4</span>

In [34]:
#the difference between the both groups is small
control = experiment_data[experiment_data['group'] == 'control']
treatment = experiment_data[experiment_data['group'] == 'treatment']

control_converted = round(control['converted'].sum() * 100/control['converted'].count(), 3)
treatment_converted = round(treatment['converted'].sum() * 100/treatment['converted'].count(), 3)

lift = round(control_converted - treatment_converted, 3)

In [35]:
#the actual difference ratio
lift

0.144

In [36]:
#buiding a chi test for weekly effect
observed_values2 = pd.crosstab(experiment_data['group'], experiment_data['converted']).values
observed_values2

array([[126073,  17220],
       [126372,  17025]], dtype=int64)

In [37]:
chi2_statistics, p_value, dof, expected_values = chi2_contingency(observed_values2, correction=False)
print(chi2_statistics, p_value)

1.426794609399621 0.23228827305833816


In [38]:
#testing hypotheses again
h_0 = 'no relationship'
h_1 = 'relationship exists'
if chi2_statistics > a:
    print(h_1, ':', 'chi2', chi2_statistics, 'p_value', p_value)
elif chi2_statistics < a:
    print(h_0, ':', 'chi2', chi2_statistics, 'p_value', p_value)

relationship exists : chi2 1.426794609399621 p_value 0.23228827305833816


<span style='font-size: 25px'>First week exposure...</span>

The conversion ratio stays at 13.5%, with p_value of .54. 

<span style='font-size: 25px'>Second week exposure...</span>


The conversion ration fluctuates at 12%, with p_value of .43


<span style='font-size: 25px'>Third week exposure...</span>

The conversion ration fluctuates stay at 13.3% with p_value of .29 (The reduce in probability of seeing a difference has dropped significantly)

<span style='font-size: 25px'>Fourth week exposure...</span>

The conversion ration fluctuates stay at 14.4% with p_value of .23 

<h2>The conversion ration fluctuates within 1%, with p_value decreasing with every week. That tells us, that the most conversions occurred during the first week, and the speed of conversion was slowing down throughout the whole experiment</h2>


<span style='font-size: 30px; color:green;'>With this we can say that there is indeed a difference between ratios of control and treatment groups, the difference proved to equal roughly 15 percent!!!</span>